In [0]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pandas as pd
from pyspark.sql.functions import col, min, max

In [0]:
spark = SparkSession.builder.appName("MyDataProcessingJob").getOrCreate()

In [0]:
trips_raw = spark.read.table("hive_metastore.metrobikes.trips")
#trips_raw.show(10) 

In [0]:
kiosks_raw = spark.read.table("hive_metastore.metrobikes.kiosk_locations")
#kiosks_raw.show(10) 

In [0]:
sensors_raw = spark.read.table("hive_metastore.metrobikes.sensors")
#sensors_raw.show(10)  

In [0]:
traffic_raw = spark.read.table("hive_metastore.metrobikes.traffic_counts")
#traffic_raw.show(10) 

Removing whitespaces from column names.

In [0]:
def remove_whitespace(df):
    column_names = df.columns
    column_names_no_spaces = [s.replace(' ', '') for s in column_names]
    column_name_mapping = dict(zip(column_names, column_names_no_spaces))
    for old_col, new_col in column_name_mapping.items():
        df = df.withColumnRenamed(old_col, new_col)
    return df

In [0]:
trips_raw = remove_whitespace(trips_raw)

In [0]:
trips_raw.columns

['TripID',
 'MembershiporPassType',
 'BicycleID',
 'BikeType',
 'CheckoutDatetime',
 'CheckoutDate',
 'CheckoutTime',
 'CheckoutKioskID',
 'CheckoutKiosk',
 'ReturnKioskID',
 'ReturnKiosk',
 'TripDurationMinutes',
 'Month',
 'Year']

In [0]:
kiosks_raw = remove_whitespace(kiosks_raw)

In [0]:
kiosks_raw.columns

['KioskID',
 'KioskName',
 'KioskStatus',
 'Location',
 'Address',
 'AlternateName',
 'CityAssetNumber',
 'PropertyType',
 'NumberofDocks',
 'PowerType',
 'FootprintLength',
 'FootprintWidth',
 'Notes',
 'CouncilDistrict',
 'Image',
 'ModifiedDate']

In [0]:
sensors_raw = remove_whitespace(sensors_raw)
sensors_raw.columns

['READER_ID',
 'ATD_SENSOR_ID',
 'KITS_ID',
 'ATD_LOCATION_ID',
 'MODIFIED_DATE',
 'SENSOR_STATUS',
 'TURN_ON_DATE',
 'SENSOR_TYPE',
 'SENSOR_MFG',
 'COA_INTERSECTION_ID',
 'PRIMARY_ST_SEGMENT_ID',
 'CROSS_ST_SEGMENT_ID',
 'LANDMARK',
 'PRIMARY_ST_AKA',
 'CROSS_ST_AKA',
 'SIGNAL_ENG_AREA',
 'COUNCIL_DISTRICT',
 'JURISDICTION',
 'LOCATION_TYPE',
 'LOCATION_NAME',
 'PRIMARY_ST',
 'CROSS_ST',
 'PRIMARY_ST_BLOCK',
 'COUNTY',
 'CROSS_ST_BLOCK',
 'IP_COMM_STATUS',
 'COMM_STATUS_DATETIME_UTC',
 'LOCATION_LATITUDE',
 'LOCATION_LONGITUDE',
 'SourceDBID',
 'LOCATION',
 'PRIMARY_ST_SEGMENT_DISPLAY_NAME',
 'CROSS_ST_SEGMENT_DISPLAY_NAME',
 'JURISDICTION_LABEL',
 'CORRIDOR_NAME']

In [0]:
traffic_raw = remove_whitespace(traffic_raw)
traffic_raw.columns

['RowID',
 'DetectorID',
 'KITSID',
 'ReadDate',
 'IntersectionName',
 'Lane',
 'Volume',
 'Occupancy',
 'Speed',
 'Month',
 'Day',
 'Year',
 'Hour',
 'Minute',
 'DayofWeek',
 'TimeBin',
 'Direction']

Tranforming the data, creating fact and dimension tables

In [0]:
start_date = datetime.strptime("2021-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-12-01", "%Y-%m-%d")
date_list = pd.date_range(start_date, end_date, freq='D')
date_list_iso = date_list.strftime("%Y-%m-%d")
date_df = pd.DataFrame()
date_df['DateISO'] = pd.to_datetime(date_list_iso)
date_df["Year"] = date_df['DateISO'].dt.year.apply(int)
date_df["YearName"] = date_df['DateISO'].dt.year.apply(str)
date_df["MonthName"] = date_df['DateISO'].dt.month.apply(str).str.zfill(2)
date_df["MonthNumber"] = date_df["MonthName"].apply(int)
date_df["DayName"] = date_df['DateISO'].dt.day.apply(str).str.zfill(2)
date_df["DayNumber"] = date_df["DayName"].apply(int)
date_df["DateKey"] = (date_df["YearName"] + date_df["MonthName"] + date_df["DayName"]).apply(int)
date_df = date_df[['DateKey', 'DateISO', 'Year', 'MonthNumber', 'MonthName', 'DayNumber', 'DayName']]

In [0]:
date_dim = spark.createDataFrame(date_df)

In [0]:
date_dim = date_dim.withColumn("DateISO", col("DateISO").cast("date"))

In [0]:
date_dim.show(10)

+--------+----------+----+-----------+---------+---------+-------+
| DateKey|   DateISO|Year|MonthNumber|MonthName|DayNumber|DayName|
+--------+----------+----+-----------+---------+---------+-------+
|20210101|2021-01-01|2021|          1|       01|        1|     01|
|20210102|2021-01-02|2021|          1|       01|        2|     02|
|20210103|2021-01-03|2021|          1|       01|        3|     03|
|20210104|2021-01-04|2021|          1|       01|        4|     04|
|20210105|2021-01-05|2021|          1|       01|        5|     05|
|20210106|2021-01-06|2021|          1|       01|        6|     06|
|20210107|2021-01-07|2021|          1|       01|        7|     07|
|20210108|2021-01-08|2021|          1|       01|        8|     08|
|20210109|2021-01-09|2021|          1|       01|        9|     09|
|20210110|2021-01-10|2021|          1|       01|       10|     10|
+--------+----------+----+-----------+---------+---------+-------+
only showing top 10 rows



In [0]:
start_time = datetime.strptime("00:00:00", "%H:%M:%S")
end_time = datetime.strptime("23:59:59", "%H:%M:%S")
time_list = pd.date_range(start_time, end_time, freq='S')

time_df = pd.DataFrame()
time_df['TimeISO'] = time_list
time_df["HourNumber"] = (time_df['TimeISO'].dt.hour).astype(int)
time_df["HourName"] = time_df['TimeISO'].dt.hour.apply(str).str.zfill(2)
time_df["MinuteNumber"] = (time_df['TimeISO'].dt.minute).astype(int)
time_df["MinuteName"] = time_df['TimeISO'].dt.minute.apply(str).str.zfill(2)
time_df["SecondNumber"] = (time_df['TimeISO'].dt.second).astype(int)
time_df["SecondName"] = time_df['TimeISO'].dt.second.apply(str).str.zfill(2)
time_df["TimeKey"] = time_df["HourName"] + time_df["MinuteName"] + time_df["SecondName"]

time_df = time_df[['TimeKey', 'HourNumber', 'HourName', 'MinuteNumber', 'MinuteName', 'SecondNumber', 'SecondName']]

In [0]:
time_dim = spark.createDataFrame(time_df)

In [0]:
!pip install shapely

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
def extract_location(r):
    from shapely import wkt
    r.dropna()
    latitude_list = []
    longitude_list = []
    for s in r:
        if s is not None:
            point = wkt.loads(s)
            latitude_list.append(point.y)
            longitude_list.append(point.x)

    df = pd.DataFrame({'PointString': r,
                   'Latitude': latitude_list,
                   'Longitude': longitude_list})
    return df

In [0]:
df = sensors_raw.toPandas()
df.dropna(subset=['LOCATION'], inplace=True)
locationsdf = extract_location(df["LOCATION"])
locationsdf.head()

,PointString,Latitude,Longitude
0,POINT (-97.796646 30.469118),30.469118,-97.796646
1,POINT (-97.751737 30.244513),30.244513,-97.751737
2,POINT (-97.781705 30.243875),30.243875,-97.781705
3,POINT (-97.717606 30.402287),30.402287,-97.717606
4,POINT (-97.78566 30.371674),30.371674,-97.785660


In [0]:
locationsdf.dtypes

PointString     object
Latitude       float64
Longitude      float64
dtype: object

In [0]:
def string_to_loc(original_loc_column):
    latitude_list = []
    longitude_list = []
    for s in original_loc_column:
        coordinates_str = s.strip('()')
        latitude_str, longitude_str = coordinates_str.split(", ")
        latitude_list.append(float(latitude_str))
        longitude_list.append(float(longitude_str))
    df = pd.DataFrame({'PointString': original_loc_column,
                   'Latitude': latitude_list,
                   'Longitude': longitude_list})
    return df

In [0]:
df2 = kiosks_raw.toPandas()
df2.dropna(subset=['Location'], inplace=True)
locationsdf = locationsdf.append(string_to_loc(df2['Location']), ignore_index=True)

/root/.ipykernel/589531/command-3471958176486951-1161582401:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  locationsdf = locationsdf.append(string_to_loc(df2['Location']), ignore_index=True)


In [0]:
locationsdf.dtypes

PointString     object
Latitude       float64
Longitude      float64
dtype: object

In [0]:
locationsdf["LocationKey"] = np.arange(1, len(locationsdf) + 1).tolist()

In [0]:
locationsdf.tail(20)

,PointString,Latitude,Longitude,LocationKey
140,"(30.283, -97.7375)",30.283000,-97.737500,141
141,"(30.2856, -97.7335)",30.285600,-97.733500,142
142,"(30.29068, -97.74292)",30.290680,-97.742920,143
143,"(30.28728, -97.74495)",30.287280,-97.744950,144
144,"(30.27059, -97.74441)",30.270590,-97.744410,145
145,"(30.25495, -97.74755)",30.254950,-97.747550,146
146,"(30.27489, -97.76483)",30.274890,-97.764830,147
147,"(30.27024, -97.73578)",30.270240,-97.735780,148
148,"(30.26888, -97.72431)",30.268880,-97.724310,149
149,"(30.26969, -97.71873)",30.269690,-97.718730,150


In [0]:
location_dim =  spark.createDataFrame(locationsdf[["LocationKey", "Latitude", "Longitude"]])

In [0]:
trips_df = trips_raw.toPandas()
trips_df.dtypes

TripID                           int64
MembershiporPassType            object
BicycleID                      float64
BikeType                        object
CheckoutDatetime                object
CheckoutDate                    object
CheckoutTime            datetime64[ns]
CheckoutKioskID                 object
CheckoutKiosk                   object
ReturnKioskID                   object
ReturnKiosk                     object
TripDurationMinutes              int64
Month                            int64
Year                             int64
dtype: object

In [0]:
trips_df.replace([np.inf, -np.inf], np.nan, inplace=True)
trips_df.dropna(subset=['BicycleID'], inplace=True)
trips_df["BicycleID"] = trips_df["BicycleID"].astype(int)

In [0]:
dfg = trips_df.groupby(by=["BicycleID", "BikeType"]).size().to_frame().reset_index()
dfg = dfg.iloc[: , :-1]
bike_dim =  spark.createDataFrame(dfg)

Fact Tables

In [0]:
trips_df.head()

,TripID,MembershiporPassType,BicycleID,BikeType,CheckoutDatetime,CheckoutDate,CheckoutTime,CheckoutKioskID,CheckoutKiosk,ReturnKioskID,ReturnKiosk,TripDurationMinutes,Month,Year
0,10368110,Walk Up,429,classic,06/08/2016 02:14:15 PM,2016-06-08,2023-12-08 14:14:15,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,44,6,2016
1,10368119,Walk Up,897,classic,06/08/2016 02:14:49 PM,2016-06-08,2023-12-08 14:14:49,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,44,6,2016
2,10368132,Walk Up,712,classic,06/08/2016 02:16:00 PM,2016-06-08,2023-12-08 14:16:00,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,39,6,2016
3,10368143,Walk Up,362,classic,06/08/2016 02:16:37 PM,2016-06-08,2023-12-08 14:16:37,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,42,6,2016
4,10368151,Walk Up,22,classic,06/08/2016 02:17:26 PM,2016-06-08,2023-12-08 14:17:26,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,38,6,2016


In [0]:
filtered_trips_df = trips_df[trips_df['Year'].isin([2021, 2022, 2023])]
filtered_trips_df.dropna(subset=['CheckoutDatetime'], inplace=True)
filtered_trips_df.dropna(subset=['CheckoutDate'], inplace=True)
filtered_trips_df.dropna(subset=['CheckoutTime'], inplace=True)
filtered_trips_df.dropna(subset=['CheckoutKioskID'], inplace=True)
filtered_trips_df.dropna(subset=['ReturnKioskID'], inplace=True)
filtered_trips_df.dropna(subset=['TripDurationMinutes'], inplace=True)
filtered_trips_df.dropna(subset=['TripID'], inplace=True)
filtered_trips_df.dropna(subset=['MembershiporPassType'], inplace=True)
filtered_trips_df.dropna(subset=['BicycleID'], inplace=True)
filtered_trips_df


/root/.ipykernel/589531/command-3471958176486960-1236736235:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df.dropna(subset=['CheckoutDatetime'], inplace=True)
/root/.ipykernel/589531/command-3471958176486960-1236736235:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df.dropna(subset=['CheckoutDate'], inplace=True)
/root/.ipykernel/589531/command-3471958176486960-1236736235:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,TripID,MembershiporPassType,BicycleID,BikeType,CheckoutDatetime,CheckoutDate,CheckoutTime,CheckoutKioskID,CheckoutKiosk,ReturnKioskID,ReturnKiosk,TripDurationMinutes,Month,Year
59,30124963,Student Membership,17751,electric,07/05/2023 03:11:38 PM,2023-07-05,2023-12-08 15:11:38,3798.0,21st/Speedway @ PCL,3798.0,23rd/Pearl,6,7,2023
614,25800691,Student Membership,21829,electric,12/12/2021 04:43:52 PM,2021-12-12,2023-12-08 16:43:52,2572.0,Barton Springs Pool,3684.0,Cesar Chavez/Congress,40,12,2021
1018,30125060,Explorer,18939,electric,07/05/2023 03:24:30 PM,2023-07-05,2023-12-08 15:24:30,3619.0,6th/Congress,3619.0,21st/Speedway @ PCL,181,7,2023
1099,30125063,Explorer,21849,electric,07/05/2023 03:24:46 PM,2023-07-05,2023-12-08 15:24:46,3619.0,6th/Congress,3619.0,6th/Trinity,255,7,2023
1389,25800774,Student Membership,21803,electric,12/12/2021 04:56:24 PM,2021-12-12,2023-12-08 16:56:24,2548.0,Guadalupe/West Mall @ University Co-op,3795.0,Dean Keeton/Whitis,10,12,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085958,31381259,Local31,19271,electric,10/06/2023 06:06:41 PM,2023-10-06,2023-12-08 18:06:41,2501.0,5th/Bowie,4061,Lakeshore/Austin Hostel,22,10,2023
2085959,31735373,Pay-as-you-ride,21535,electric,10/31/2023 03:24:31 PM,2023-10-31,2023-12-08 15:24:31,2504.0,South Congress/Elizabeth,2567,Barton Springs/Bouldin @ Palmer Auditorium,69,10,2023
2085960,31601940,Local31,19376,electric,10/20/2023 09:05:58 PM,2023-10-20,2023-12-08 21:05:58,3619.0,6th/Congress,4060,Red River/Cesar Chavez @ The Fairmont,12,10,2023
2085961,31464528,Local31,21868,electric,10/11/2023 04:45:16 PM,2023-10-11,2023-12-08 16:45:16,2494.0,2nd/Congress,2571,8th/Red River,6,10,2023


In [0]:
filtered_trips_df.rename(columns={'BicycleID': 'BikeKey', 'MembershiporPassType': 'PassType'}, inplace=True)

/root/.ipykernel/589531/command-3471958176486964-3805949074:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df.rename(columns={'BicycleID': 'BikeKey', 'MembershiporPassType': 'PassType'}, inplace=True)


In [0]:
#convert datetime64[ns] to a time string
filtered_trips_df["TimeKey"] = filtered_trips_df['CheckoutTime'].dt.strftime("%H%M%S")
filtered_trips_df['CheckoutDate'] = pd.to_datetime(filtered_trips_df['CheckoutDate'])
filtered_trips_df["DateKey"] = filtered_trips_df['CheckoutDate'].dt.strftime("%Y%m%d").astype(int)

/root/.ipykernel/589531/command-3471958176486965-2929036326:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df["TimeKey"] = filtered_trips_df['CheckoutTime'].dt.strftime("%H%M%S")
/root/.ipykernel/589531/command-3471958176486965-2929036326:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df['CheckoutDate'] = pd.to_datetime(filtered_trips_df['CheckoutDate'])
/root/.ipykernel/589531/command-3471958176486965-2929036326:4: SettingWithCopyWarning: 
A value is trying to be set on a 

In [0]:
filtered_trips_df['CheckoutKioskID'] = filtered_trips_df['CheckoutKioskID'].astype(float).astype(int)
filtered_trips_df['ReturnKioskID'] = filtered_trips_df['ReturnKioskID'].astype(float).astype(int)

/root/.ipykernel/589531/command-3471958176486972-4107142944:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df['CheckoutKioskID'] = filtered_trips_df['CheckoutKioskID'].astype(float).astype(int)
/root/.ipykernel/589531/command-3471958176486972-4107142944:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trips_df['ReturnKioskID'] = filtered_trips_df['ReturnKioskID'].astype(float).astype(int)


In [0]:
filtered_trips_df.head(10)

,TripID,PassType,BikeKey,BikeType,CheckoutDatetime,CheckoutDate,CheckoutTime,CheckoutKioskID,CheckoutKiosk,ReturnKioskID,ReturnKiosk,TripDurationMinutes,Month,Year,TimeKey,DateKey
59,30124963,Student Membership,17751,electric,07/05/2023 03:11:38 PM,2023-07-05,2023-12-08 15:11:38,3798,21st/Speedway @ PCL,3798,23rd/Pearl,6,7,2023,151138,20230705
614,25800691,Student Membership,21829,electric,12/12/2021 04:43:52 PM,2021-12-12,2023-12-08 16:43:52,2572,Barton Springs Pool,3684,Cesar Chavez/Congress,40,12,2021,164352,20211212
1018,30125060,Explorer,18939,electric,07/05/2023 03:24:30 PM,2023-07-05,2023-12-08 15:24:30,3619,6th/Congress,3619,21st/Speedway @ PCL,181,7,2023,152430,20230705
1099,30125063,Explorer,21849,electric,07/05/2023 03:24:46 PM,2023-07-05,2023-12-08 15:24:46,3619,6th/Congress,3619,6th/Trinity,255,7,2023,152446,20230705
1389,25800774,Student Membership,21803,electric,12/12/2021 04:56:24 PM,2021-12-12,2023-12-08 16:56:24,2548,Guadalupe/West Mall @ University Co-op,3795,Dean Keeton/Whitis,10,12,2021,165624,20211212
5205,25800811,Student Membership,16333,electric,12/12/2021 05:00:20 PM,2021-12-12,2023-12-08 17:00:20,3686,Sterzing/Barton Springs,3294,6th/Lavaca,15,12,2021,170020,20211212
7043,30125186,Student Membership,21558,electric,07/05/2023 03:38:39 PM,2023-07-05,2023-12-08 15:38:39,7188,22nd/Pearl,7188,5th/Bowie,7,7,2023,153839,20230705
9655,25800817,3-Day Weekender,21523,electric,12/12/2021 05:01:23 PM,2021-12-12,2023-12-08 17:01:23,3621,3rd/Nueces,2496,8th/Congress,78,12,2021,170123,20211212
9939,25801020,Local365,21419,electric,12/12/2021 05:32:42 PM,2021-12-12,2023-12-08 17:32:42,2563,Rainey/Davis,2547,21st/Guadalupe,11,12,2021,173242,20211212
9981,25801023,Local31,23277,electric,12/12/2021 05:34:02 PM,2021-12-12,2023-12-08 17:34:02,4061,Lakeshore/Austin Hostel,2575,Riverside/South Lamar,51,12,2021,173402,20211212


In [0]:
kiosks_df = kiosks_raw.toPandas()
kiosks_df.dtypes

KioskID              int64
KioskName           object
KioskStatus         object
Location            object
Address             object
AlternateName       object
CityAssetNumber    float64
PropertyType        object
NumberofDocks      float64
PowerType           object
FootprintLength    float64
FootprintWidth     float64
Notes               object
CouncilDistrict      int64
Image               object
ModifiedDate        object
dtype: object

In [0]:
kiosks_df = pd.merge(kiosks_df, locationsdf, left_on='Location', right_on='PointString', how='left')
kiosks_df

,KioskID,KioskName,KioskStatus,Location,Address,AlternateName,CityAssetNumber,PropertyType,NumberofDocks,PowerType,FootprintLength,FootprintWidth,Notes,CouncilDistrict,Image,ModifiedDate,PointString,Latitude,Longitude,LocationKey
0,11,22nd 1/2 & Rio Grande,active,"(30.2862, -97.74516)",710 W. 22 1/2 St,None,NaN,sidewalk,4.0,solar,NaN,NaN,Kiosk ID & footprint length and width to be re...,9,None,11/04/2021 02:24:00 PM,NaN,NaN,NaN,NaN
1,111,23rd & San Gabriel,active,"(30.2874, -97.7478)",915 W 23rd St,None,NaN,paid_parking,13.0,solar,NaN,NaN,Kiosk ID and Footprint length & width to be re...,9,None,11/04/2021 02:15:00 PM,NaN,NaN,NaN,NaN
2,1001,OFFICE/Main/Shop/Repair,closed,"(30.27186, -97.73997)",1000 Brazos,None,NaN,,NaN,,NaN,NaN,This is the testing dock/station located at BS...,1,None,03/04/2022 09:58:00 AM,NaN,NaN,NaN,NaN
3,1002,6th & Navasota St.,closed,"(30.26383, -97.72864)",1308 W. 6th St.,None,NaN,None,NaN,None,NaN,NaN,None,3,None,01/04/2021 12:00:00 AM,NaN,NaN,NaN,NaN
4,1003,8th & Guadalupe,closed,"(30.27106, -97.74563)",800 Guadalupe St.,None,NaN,None,NaN,None,NaN,NaN,None,9,None,01/04/2021 12:00:00 AM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,4061,Lakeshore @ Austin Hostel,active,"(30.24472, -97.72336)",2200 S Lakeshore Blvd,None,32692.0,nonmetered_parking,15.0,solar,45.0,5.0,None,3,None,01/04/2021 12:00:00 AM,NaN,NaN,NaN,NaN
98,4062,Lakeshore & Pleasant Valley,active,"(30.24258, -97.71726)",2525 South Lakeshore Boulevard,None,32711.0,nonmetered_parking,18.0,solar,55.0,5.0,None,3,None,01/04/2021 12:00:00 AM,NaN,NaN,NaN,NaN
99,4699,East 5th/Shady @ Eastside Bus Plaza,active,"(30.25212, -97.69807)",5104 East 5th. Street,None,32522.0,sidewalk,10.0,solar,15.0,10.0,None,3,None,07/27/2021 08:25:00 AM,NaN,NaN,NaN,NaN
100,4879,16th/San Antonio,active,"(30.27924, -97.74371)",1601 San Antonio Street,None,16739.0,paid_parking,12.0,solar,40.0,5.0,None,9,None,03/12/2021 01:15:00 PM,NaN,NaN,NaN,NaN


In [0]:
trips_kiosks_df = pd.merge(filtered_trips_df, kiosks_df, left_on='CheckoutKioskID', right_on='KioskID', how='left')

In [0]:
trips_kiosks_df.dropna(subset=['KioskID'], inplace=True)

In [0]:
trips_kiosks_loc_df = pd.merge(trips_kiosks_df, locationsdf, left_on='Location', right_on='PointString', how='left')

In [0]:
trips_kiosks_loc_df.head(25)

,TripID,PassType,BikeKey,BikeType,CheckoutDatetime,CheckoutDate,CheckoutTime,CheckoutKioskID,CheckoutKiosk,ReturnKioskID,ReturnKiosk,TripDurationMinutes,Month,Year,TimeKey,DateKey,KioskID,KioskName,KioskStatus,Location,Address,AlternateName,CityAssetNumber,PropertyType,NumberofDocks,PowerType,FootprintLength,FootprintWidth,Notes,CouncilDistrict,Image,ModifiedDate,PointString,Latitude,Longitude,LocationKey
0,30124963,Student Membership,17751,electric,07/05/2023 03:11:38 PM,2023-07-05,2023-12-08 15:11:38,3798,21st/Speedway @ PCL,3798,23rd/Pearl,6,7,2023,151138,20230705,3798.0,21st & Speedway @PCL,active,"(30.283, -97.7375)",E. 21st St.,None,32675.0,sidewalk,22.0,solar,30.0,7.5,On UT property-2 sided,9.0,None,01/04/2021 12:00:00 AM,"(30.283, -97.7375)",30.283000,-97.737500,141
1,25800691,Student Membership,21829,electric,12/12/2021 04:43:52 PM,2021-12-12,2023-12-08 16:43:52,2572,Barton Springs Pool,3684,Cesar Chavez/Congress,40,12,2021,164352,20211212,2572.0,Barton Springs Pool,active,"(30.26452, -97.7712)",2200 William Barton Dr.,None,16758.0,parkland,13.0,solar,40.0,5.0,None,8.0,None,01/04/2021 12:00:00 AM,"(30.26452, -97.7712)",30.264520,-97.771200,108
2,30125060,Explorer,18939,electric,07/05/2023 03:24:30 PM,2023-07-05,2023-12-08 15:24:30,3619,6th/Congress,3619,21st/Speedway @ PCL,181,7,2023,152430,20230705,3619.0,6th & Congress,active,"(30.26822, -97.74285)",600 Congress Ave.,Congress & 6th Street,16743.0,sidewalk,11.0,solar,30.0,5.0,remeasure,9.0,None,01/04/2021 12:00:00 AM,"(30.26822, -97.74285)",30.268220,-97.742850,126
3,30125063,Explorer,21849,electric,07/05/2023 03:24:46 PM,2023-07-05,2023-12-08 15:24:46,3619,6th/Congress,3619,6th/Trinity,255,7,2023,152446,20230705,3619.0,6th & Congress,active,"(30.26822, -97.74285)",600 Congress Ave.,Congress & 6th Street,16743.0,sidewalk,11.0,solar,30.0,5.0,remeasure,9.0,None,01/04/2021 12:00:00 AM,"(30.26822, -97.74285)",30.268220,-97.742850,126
4,25800774,Student Membership,21803,electric,12/12/2021 04:56:24 PM,2021-12-12,2023-12-08 16:56:24,2548,Guadalupe/West Mall @ University Co-op,3795,Dean Keeton/Whitis,10,12,2021,165624,20211212,2548.0,UT West Mall @ Guadalupe,active,"(30.28576, -97.74181)",2242 Guadalupe St.,None,16748.0,paid_parking,15.0,solar,45.0,5.0,"in buffer area parking, check dock #",9.0,None,01/04/2021 12:00:00 AM,"(30.28576, -97.74181)",30.285760,-97.741810,93
5,25800811,Student Membership,16333,electric,12/12/2021 05:00:20 PM,2021-12-12,2023-12-08 17:00:20,3686,Sterzing/Barton Springs,3294,6th/Lavaca,15,12,2021,170020,20211212,3686.0,Sterzing at Barton Springs,active,"(30.26406, -97.76385)",1825 Barton Springs Rd,None,32592.0,undetermined_parking,13.0,solar,40.0,5.0,None,5.0,None,01/04/2021 12:00:00 AM,"(30.26406, -97.76385)",30.264060,-97.763850,132
6,25800817,3-Day Weekender,21523,electric,12/12/2021 05:01:23 PM,2021-12-12,2023-12-08 17:01:23,3621,3rd/Nueces,2496,8th/Congress,78,12,2021,170123,20211212,3621.0,Nueces & 3rd,active,"(30.26697, -97.74929)",311 Nueces St.,None,16742.0,paid_parking,11.0,solar,35.0,5.0,None,9.0,None,01/04/2021 12:00:00 AM,"(30.26697, -97.74929)",30.266970,-97.749290,127
7,25801020,Local365,21419,electric,12/12/2021 05:32:42 PM,2021-12-12,2023-12-08 17:32:42,2563,Rainey/Davis,2547,21st/Guadalupe,11,12,2021,173242,20211212,2563.0,Rainey/Driskill,active,"(30.260814, -97.738086)",698 Davis St,None,16751.0,paid_parking,12.0,non-metered,40.0,5.0,None,9.0,None,03/04/2022 08:26:00 AM,"(30.260814, -97.738086)",30.260814,-97.738086,99
8,25801023,Local31,23277,electric,12/12/2021 05:34:02 PM,2021-12-12,2023-12-08 17:34:02,4061,Lakeshore/Austin Hostel,2575,Riverside/South Lamar,51,12,2021,173402,20211212,4061.0,Lakeshore @ Austin Hostel,active,"(30.24472, -97.72336)",2200 S Lakeshore Blvd,None,32692.0,nonmetered_parking,15.0,solar,45.0,5.0,None,3.0,None,01/04/2021 12:00:00 AM,"(30.24472, -97.72336)",30.244720,-97.723360,156
9,25801106,Local365,19207,electric,12/12/2021 05:48:11 PM,2021-12-12,2023-12-08 17:48:11,3798,21st/Speedway @ PCL,3793,28th/R

In [0]:
trips_kiosks_loc_df.columns

Index(['TripID', 'PassType', 'BikeKey', 'BikeType', 'CheckoutDatetime',
       'CheckoutDate', 'CheckoutTime', 'CheckoutKioskID', 'CheckoutKiosk',
       'ReturnKioskID', 'ReturnKiosk', 'TripDurationMinutes', 'Month', 'Year',
       'TimeKey', 'DateKey', 'KioskID', 'KioskName', 'KioskStatus', 'Location',
       'Address', 'AlternateName', 'CityAssetNumber', 'PropertyType',
       'NumberofDocks', 'PowerType', 'FootprintLength', 'FootprintWidth',
       'Notes', 'CouncilDistrict', 'Image', 'ModifiedDate', 'PointString',
       'Latitude', 'Longitude', 'LocationKey'],
      dtype='object')

In [0]:
trips_kiosks_loc_df = trips_kiosks_loc_df[["DateKey", "TimeKey", "LocationKey", "BikeKey", "TripID", "TripDurationMinutes", "PassType"]]

In [0]:
trips_kiosks_loc_df.isna().sum()

DateKey                0
TimeKey                0
LocationKey            0
BikeKey                0
TripID                 0
TripDurationMinutes    0
dtype: int64

In [0]:
fact_bike_trip =  spark.createDataFrame(trips_kiosks_loc_df)

In [0]:
fact_bike_trip.write.format("delta").save("/hive_metastore/path/to/your/new/delta/folderv")
spark.sql("CREATE TABLE metrobikes.fact_bike_trip USING delta LOCATION '/hive_metastore/path/to/your/new/delta/folder'")

DataFrame[]

In [0]:
location_dim.write.format("delta").save("/hive_metastore/path/to/your/new/delta/folder")
spark.sql("CREATE TABLE metrobikes.location_dim USING delta LOCATION '/hive_metastore/path/to/your/new/delta/folder'")

DataFrame[]

In [0]:
time_dim.write.format("delta").save("/hive_metastore/path/to/your/new/delta/folder")
spark.sql("CREATE TABLE metrobikes.time_dim USING delta LOCATION '/hive_metastore/path/to/your/new/delta/folder'")

DataFrame[]

In [0]:
bike_dim.write.format("delta").save("/hive_metastore/path/to/your/new/delta/folder")
spark.sql("CREATE TABLE metrobikes.bike_dim USING delta LOCATION '/hive_metastore/path/to/your/new/delta/folder'")

DataFrame[]

In [0]:
date_dim.write.format("delta").save("/hive_metastore/path/to/your/new/delta/folder")
spark.sql("CREATE TABLE metrobikes.date_dim USING delta LOCATION '/hive_metastore/path/to/your/new/delta/folder'")

DataFrame[]